In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
# from textblob import TextBlob

import datetime as dt
import pandas as pd
import numpy as np
import warnings
import string

stop_words = []
stop_words = list(set(stopwords.words('english')))
warnings.filterwarnings('ignore')
punctuation = string.punctuation

In [52]:
id_column = "id"
missing_token = " UNK "

df = pd.read_csv("train.csv", parse_dates=["project_submitted_datetime"])
# test = pd.read_csv("test.csv", parse_dates=["project_submitted_datetime"])
# rc = pd.read_csv("resources.csv").fillna(missing_token)

#df = pd.concat([train, test], axis=0) 

In [53]:
df.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [54]:
# Generate the Category variables 

df["lsLangLit"]=df["project_subject_categories"].str.contains("Literacy & Language")
df["lsMathSci"]=df["project_subject_categories"].str.contains("Math & Science")
df["lsAppLearn"]=df["project_subject_categories"].str.contains("Applied Learning")
df["lsHealth"]=df["project_subject_categories"].str.contains("Health & Sports")
df["lsSpNeeds"]=df["project_subject_categories"].str.contains("Special Needs")
df["lsHistCiv"]=df["project_subject_categories"].str.contains("History & Civics")
df["lsMusArts"]=df["project_subject_categories"].str.contains("Music & The Arts")
df["lsWarm"]=df["project_subject_categories"].str.contains("Warmth, Care & Hunger")


In [55]:
# fillup empty values with missing token 
df['project_essay_3'] = df['project_essay_3'].fillna(missing_token)
df['project_essay_4'] = df['project_essay_4'].fillna(missing_token)

# extract length of each essay and title
df["essay1_len"] = df['project_essay_1'].apply(len)
df["essay2_len"] = df['project_essay_2'].apply(len)
df["essay3_len"] = df['project_essay_3'].apply(len)
df["essay4_len"] = df['project_essay_4'].apply(len)
df["title_len"] = df['project_title'].apply(len)

In [56]:
df[['essay1_len', 'essay2_len', 'essay3_len', 'essay4_len', 'title_len']].head(10)

,essay1_len,essay2_len,essay3_len,essay4_len,title_len
0,967,805,5,5,24
1,587,639,5,5,22
2,761,546,5,5,21
3,1201,1209,5,5,72
4,451,556,5,5,48
5,492,737,5,5,37
6,789,931,5,5,21
7,458,629,5,5,36
8,573,774,5,5,37
9,624,710,5,5,27


In [57]:
# combine the project essays to create a complete essay text
df['text'] = df.apply(lambda row: ' '.join([str(row['project_essay_1']), 
                                            str(row['project_essay_2']), 
                                            str(row['project_essay_3']), 
                                            str(row['project_essay_4'])]), axis=1)

# extract features from text
df['char_count'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)
df['punctuation_count'] = df['text'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 
df['title_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df['upper_case_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df['stopword_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words]))

In [58]:
df[['char_count', 'word_count', 'word_density', 'punctuation_count', 'title_word_count', 'upper_case_word_count', 'stopword_count']].head(10)

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,stopword_count
0,1785,314,5.666667,40,21,7,151
1,1239,192,6.419689,38,15,5,79
2,1320,236,5.569620,26,26,6,103
3,2423,388,6.228792,77,31,6,188
4,1020,187,5.425532,15,13,2,98
5,1242,194,6.369231,54,21,5,88
6,1733,282,6.123675,43,23,5,127
7,1100,184,5.945946,45,16,2,75
8,1360,239,5.666667,31,11,6,111
9,1347,240,5.589212,36,16,10,114


In [59]:
temp=df.groupby('project_title')['project_is_approved'].agg(['sum','count'])
temp['approval_rate']=(temp['sum']*100)/temp['count']
temp.columns=['# of projects approved','# of total projects','Approval rate']
temp=temp.sort_values(by='# of total projects',ascending=False)
temp=temp.iloc[0:20]
temp

,# of projects approved,# of total projects,Approval rate
project_title,,,
Flexible Seating,298,377,79.045093
Wiggle While You Work,137,150,91.333333
Wiggle While We Work,136,149,91.275168
Can You Hear Me Now?,135,144,93.750000
Wiggle While You Work!,122,134,91.044776
Wiggle While We Work!,116,123,94.308943
"We Like to Move It, Move It!",85,100,85.000000
Listen Up!,77,83,92.771084
Technology in the Classroom,69,79,87.341772


In [70]:
# Cleaning up variables to get features vs. target(s)
df2 = df # Preserve original dataframe

if 'project_is_approved'in df:
    y1 = df['project_is_approved'] # Creates the approval target 
    del df['project_is_approved']

y2 = df[['lsLangLit','lsMathSci','lsHealth','lsSpNeeds','lsHistCiv','lsMusArts','lsWarm']]# creates a target matrix of the categories

# Remove the target category variables
if 'lsLangLit'in df:
    del df['lsLangLit']
    del df['lsMathSci']
    del df['lsHealth']
    del df['lsSpNeeds']
    del df['lsHistCiv']
    del df['lsMusArts']
    del df['lsWarm']
    
if 'project_subject_categories'in df:
    y3 = df['project_subject_categories'] # Creates the approval target if we're using the full listing and not one-hot encoded 
    del df['project_subject_categories'] # Remove the categories from the training set
    del df['project_subject_subcategories'] # Remove the subcategories, since they are dependent on the categories

,lsLangLit,lsMathSci,lsHealth,lsSpNeeds,lsHistCiv,lsMusArts,lsWarm
0,True,False,False,False,False,False,False
1,False,False,True,False,False,True,False
2,True,True,False,False,False,False,False
3,False,False,True,False,False,False,False
4,False,False,True,False,False,False,False


In [81]:
# Splitting here to have train vs. test data 

train=df.sample(frac=0.2,random_state=200) #Used a small sample to allow processing (encountered memory issues as at frac=0.5 and higher)

In [85]:
# Creating the Sparse matrix to possibly use KNN?

# Started with code snippet from https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab and transform text documents
new_text = vectorizer.fit_transform(train['text'])

# summarize
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)

# summarize encoded vector
print(new_text.shape)
print(new_text.toarray())


(36416, 38510)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [ ]:
# Use KNN to classify into category?
# Because some instances have multiple categories, we may need to run a series of classifications as "this" and "not this" for each category
# We could also consider using the Category itself, without extracting the different values into the one-hot encoding, essentially creating a very long list of compound categories.
